In [ ]:
# Importing Libraries
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
#pip install -U langchain-community


In [ ]:
#!python3 -m ensurepip --upgrade
#!python3 -m pip install --upgrade pip
#!python3 -m pip install langchain langchain-core langchain-google-genai

In [ ]:
# Connect MySQL database
host = 'host-address'
port = '3306'
username = 'root'
password = 'Password123!'
database_schema = 'text_to_sql'

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [ ]:
from urllib.parse import quote_plus

password = quote_plus("Password123!")  # encodes @ as %40
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [ ]:
#pip install cryptography

In [ ]:
db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=1)
db.get_table_info()

In [ ]:
#pip install pymysql

In [ ]:
# Create the LLM Prompt Template
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
Remember : Only provide me the sql query dont include anything else. Provide me sql query in a single line dont add line breaks
Table Schema: {schema}
Question: {question}
SQL Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# get the schema of the database
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = 'YOUR_GOOGLE_GENAI_API_KEY')

In [ ]:
# Create the SQL query chain using the LLM and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema=lambda _:get_schema(db)) 
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
# test the SQL query chain with a sample question

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)